# Responsible AI dashboard for Time Series Forecasting
_**Orange Juice Sales Forecasting**_

Note: Time series forecasting is not yet supported by raiwidgets. This notebook uses an experimental setting to enable it. However, the API and functionality may change at any point in the future without warning.

## Contents
1. [Introduction](#introduction)
1. [Data](#data)
1. [Train](#train)
1. [Responsible AI Dashboard](#analyze)

## Introduction<a id="introduction"></a>
In this example, we use sktime to train and assess a time-series forecasting model for multiple time-series.

The examples in the follow code samples use the University of Chicago's Dominick's Finer Foods dataset to forecast orange juice sales. Dominick's was a grocery chain in the Chicago metropolitan area.

In [1]:
import pandas as pd
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split

## Data<a id="data"></a>
You are now ready to load the historical orange juice sales data. We will load the CSV file into a plain pandas DataFrame; the time column in the CSV is called _WeekStarting_, so it will be specially parsed into the datetime type.
Each row in the DataFrame holds a quantity of weekly sales for an OJ brand at a single store. The data also includes the sales price, a flag indicating if the OJ brand was advertised in the store that week, and some customer demographic information based on the store location. For historical reasons, the data also include the logarithm of the sales quantity. The Dominick's grocery data is commonly used to illustrate econometric modeling techniques where logarithms of quantities are generally preferred.    

The task is now to build a time-series model for the _Quantity_ column. It is important to note that this dataset is comprised of many individual time-series - one for each unique combination of _Store_ and _Brand_. To distinguish the individual time-series, we define the **time_series_id_features** the columns whose values determine the boundaries between time-series: 

In [2]:
time_column_name = "WeekStarting"
time_series_id_features = ["Store", "Brand"]
dataset_location = "https://raw.githubusercontent.com/Azure/azureml-examples/2fe81643865e1f4591e7734bd1a729093cafb826/v1/python-sdk/tutorials/automl-with-azureml/forecasting-orange-juice-sales/dominicks_OJ.csv"
data = pd.read_csv(dataset_location, parse_dates=[time_column_name])

# Drop the columns 'logQuantity' as it is a leaky feature.
data.drop("logQuantity", axis=1, inplace=True)

# Set up multi index with time series ID columns and time column.
data.set_index(time_series_id_features + [time_column_name], inplace=True, drop=True)
data = data.groupby(time_series_id_features).apply(lambda group: group.loc[group.name].asfreq("W-THU").interpolate())
data.sort_index(inplace=True, ascending=[True, True, True])

data.head(10)

Quantity    Advert     Price     Age60  \
Store Brand     WeekStarting                                               
2     dominicks 1990-06-14    10560.000000  1.000000  1.590000  0.232865   
                1990-06-21    10133.333333  0.833333  1.773333  0.232865   
                1990-06-28     9706.666667  0.666667  1.956667  0.232865   
                1990-07-05     9280.000000  0.500000  2.140000  0.232865   
                1990-07-12     8853.333333  0.333333  2.323333  0.232865   
                1990-07-19     8426.666667  0.166667  2.506667  0.232865   
                1990-07-26     8000.000000  0.000000  2.690000  0.232865   
                1990-08-02     6848.000000  1.000000  2.090000  0.232865   
                1990-08-09     2880.000000  0.000000  2.090000  0.232865   
                1990-08-16     2240.000000  0.000000  2.090000  0.232865   

                               COLLEGE     INCOME  Hincome150  Large HH  \
Store Brand     WeekStarting                                              
2     dominicks 1990-06-14    0.248935  10.553205    0.463887  0.103953   
                1990-06-21    0.248935  10.553205    0.463887  0.103953   
                1990-06-28    0.248935  10.553205    0.463887  0.103953   
                1990-07-05    0.248935  10.553205    0.463887  0.103953   
                1990-07-12    0.248935  10.553205    0.463887  0.103953   
                1990-07-19    0.248935  10.553205    0.463887  0.103953   
                1990-07-26    0.248935  10.553205    0.463887  0.103953   
                1990-08-02    0.248935  10.553205    0.463887  0.103953   
                1990-08-09    0.248935  10.553205    0.463887  0.103953   
                1990-08-16    0.248935  10.553205    0.463887  0.103953   

                              Minorities  WorkingWoman  SSTRDIST   SSTRVOL  \
Store Brand     WeekStarting                                                 
2     dominicks 1990-06-14       0.11428      0.303585  2.110122  1.142857   
                1990-06-21       0.11428      0.303585  2.110122  1.142857   
                1990-06-28       0.11428      0.303585  2.110122  1.142857   
                1990-07-05       0.11428      0.303585  2.110122  1.142857   
                1990-07-12       0.11428      0.303585  2.110122  1.142857   
                1990-07-19       0.11428      0.303585  2.110122  1.142857   
                1990-07-26       0.11428      0.303585  2.110122  1.142857   
                1990-08-02       0.11428      0.303585  2.110122  1.142857   
                1990-08-09       0.11428      0.303585  2.110122  1.142857   
                1990-08-16       0.11428      0.303585  2.110122  1.142857   

                              CPDIST5   CPWVOL5  
Store Brand     WeekStarting                     
2     dominicks 1990-06-14    1.92728  0.376927  
                1990-06-21    1.92728  0.376927  
                1990-06-28    1.92728  0.376927  
                1990-07-05    1.92728  0.376927  
                1990-07-12    1.92728  0.376927  
                1990-07-19    1.92728  0.376927  
                1990-07-26    1.92728  0.376927  
                1990-08-02    1.92728  0.376927  
                1990-08-09    1.92728  0.376927  
                1990-08-16    1.92728  0.376927

In [3]:
nseries = data.groupby(time_series_id_features).ngroups
print("Data contains {0} individual time-series.".format(nseries))

Data contains 249 individual time-series.


For demonstration purposes, we extract sales time-series for just a few of the stores:

In [4]:
use_stores = [2, 5, 8]
use_brands = ['tropicana', 'dominicks', 'minute.maid']
data_subset = data.loc[(use_stores, use_brands, slice(None)), :]
nseries = data_subset.groupby(time_series_id_features).ngroups
print(f"Data subset contains {nseries} individual time-series.")

Data subset contains 9 individual time-series.


### Data Splitting
We now split the data into a training and a testing set for later forecast evaluation. The test set will contain the final 20 weeks of observed sales for each time-series. The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.

In [5]:
target_column_name = "Quantity"

y = pd.DataFrame(data_subset[target_column_name])
X = data_subset.drop(columns=[target_column_name])
fh_dates = pd.DatetimeIndex(y.index.get_level_values(2).unique().sort_values().to_list()[-20:], freq='W-THU')
fh = ForecastingHorizon(fh_dates, is_relative=False)
y_train, y_test, X_train, X_test = \
    temporal_train_test_split(
        y=y,
        X=X,
        test_size=20)

C:\ProgramData\Anaconda3\envs\raitoolboxdev\lib\site-packages\sktime\datatypes\_utilities.py:41: PerformanceWarning: indexing past lexsort depth may impact performance.
  return X.loc[tuple(list(X.index[0])[:-1])].index


## Train<a id="train"></a>

You can now submit a new training run. Depending on the data and number of iterations this operation may take several minutes.
Information from each iteration will be printed to the console.  Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [6]:
# When using sktime directly we need to drop the time and time series ID columns.
model = AutoARIMA(suppress_warnings=True, error_action="ignore")
model.fit(y=y_train, X=X_train, fh=fh)
model.predict(fh=fh, X=X_test).head()

Quantity
Store Brand     WeekStarting              
2     tropicana 1992-05-21    12625.735934
                1992-05-28    12625.735934
                1992-06-04    11748.331229
                1992-06-11     5606.498295
                1992-06-18     6308.422059

In [7]:
model.predict_quantiles(fh=fh, X=X_test, alpha=[0.025, 0.975]).head()

C:\ProgramData\Anaconda3\envs\raitoolboxdev\lib\site-packages\sktime\forecasting\base\_base.py:557: UserWarning: In 0.22.0, predict_quantiles return default column level 0 name will change for univariate probabilistic quantile forecasts from 'Quantiles' to variable name. The old behaviour can be retained by setting the legacy_interface argument to True, until 0.23.0 when the legacy_interface argument will be removed.
  warn(
C:\ProgramData\Anaconda3\envs\raitoolboxdev\lib\site-packages\sktime\forecasting\base\_base.py:557: UserWarning: In 0.22.0, predict_quantiles return default column level 0 name will change for univariate probabilistic quantile forecasts from 'Quantiles' to variable name. The old behaviour can be retained by setting the legacy_interface argument to True, until 0.23.0 when the legacy_interface argument will be removed.
  warn(
C:\ProgramData\Anaconda3\envs\raitoolboxdev\lib\site-packages\sktime\forecasting\base\_base.py:557: UserWarning: In 0.22.0, predict_quantiles 

Quantiles               
                                      0.025          0.975
Store Brand     WeekStarting                              
2     tropicana 1992-05-21   -298979.489254  324230.961122
                1992-05-28   -298979.489254  324230.961122
                1992-06-04   -299856.893959  323353.556417
                1992-06-11   -305998.726893  317211.723483
                1992-06-18   -305296.803129  317913.647247

# Responsible AI Dashboard<a id="analyze"></a>

In [8]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights, FeatureMetadata

# merge X, y, and the time and time series ID features into a single DataFrame
train = X_train.join(y_train).join(X_train.index.to_frame(index=True))
test = X_test.join(y_test).join(X_test.index.to_frame(index=True))
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

feature_metadata = FeatureMetadata(
    time_series_id_features=time_series_id_features, 
    categorical_features=time_series_id_features,
    datetime_features=[time_column_name])
insights = RAIInsights(
    model=model,
    train=train,
    test=test,
    task_type="forecasting",
    target_column=target_column_name,
    feature_metadata=feature_metadata,
    forecasting_enabled=True)

ResponsibleAIDashboard(insights)

UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.
In 0.22.0, predict_quantiles return default column level 0 name will change for univariate probabilistic quantile forecasts from 'Quantiles' to variable name. The old behaviour can be retained by setting the legacy_interface argument to True, until 0.23.0 when the legacy_interface argument will be removed.
In 0.22.0, predict_quantiles return default column level 0 name will change for univariate probabilistic quantile forecasts from 'Quantiles' to variable name. The old behaviour can be retained by setting the legacy_interface argument to True, until 0.23.0 when the legacy_interface argument will be removed.
In 0.22.0, predict_quantiles return default column level 0 name will change for univariate probabilistic quantile forecasts from 'Quantiles' to variable name. The old behaviour can be retained by setting the legacy_interface argument to True, un

ResponsibleAI started at http://localhost:5000
